Edición de archivos con Awk
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/bash-for-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/bash-for-analytics/tree/master/) para explorar el repositorio usando `nbviewer`. 

## `awk`

`awk` es un comando para Unix creado para el procesamiento y reporte de archivos de texto que contengan varios campos de datos en una misma línea. Se considera que `awk` es una evolución de `sed`. Su uso básico es: 

    awk [condición] '{printf"formato", argumentos}' filename


**Algunos comandos importantes**:

* *$0*     -> Todos los campos
* *FS*     -> Separador de campos (TAB por defecto)
* *NF*     -> Número de campos en la linea actual
* *NR*     -> Número de lineas en el archivo a procesar
* *Length*     -> Longitud de la linea a procesar
* *&&*     -> Operación lógica para la intersección
* *||*     -> Operación lógica para la unión

---

En este libro se usa `awk` para realizar búsquedas (como se hace con `grep`) y editar los archivos.

Se imprimen los primeros 20 números a un archivo.

In [31]:
seq 20 > out.1

Para imprimir todas las líneas que contienen un '1' se usaría: 

La cadena /1/ indica que la línea contenga un 1, los '/' son delimitadores y la cadena {print} al final indica que se imprima la linea.

In [32]:
awk '/1/ {print}' out.1

1
10
11
12
13
14
15
16
17
18
19


Para imprimir todas las líneas que contienen un '1' al **final** se usaría:

In [33]:
awk '/1$/ {print}' out.1

1
11


El siguiente comando imprime la tercera línea:

In [34]:
awk 'NR == 3 {print}' out.1

3


El siguiente comando imprime de la linea 3 a la linea 6.

In [35]:
awk '(NR >= 3) && (NR <= 6) {print $0}' out.1

3
4
5
6


A continuación se requiere imprimir todos los registros excepto los contenidos entre la linea 3 y la linea 6:

In [36]:
awk '!((NR >= 3) && (NR <= 6)) {print $0}' out.1

1
2
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [37]:
awk '(NR > 6) || (NR < 3) {print $0}' out.1

1
2
7
8
9
10
11
12
13
14
15
16
17
18
19
20


De la línea 15 al final:

In [38]:
awk '(NR >= 15) && (NR <= NR) {print $0}' out.1

15
16
17
18
19
20


---

En el archivo generado a continuación se desea substituir la `X` por `x`:

In [1]:
cat > out.1 <<EOF
FieldA, fieldD, gieldE, FieldG
   2, X, 2X, 2XG
   2, Y, 2Y, 2YG 
   3, Y, 3Y, 3YG
   3, X, 3X, 3XG
   4, Z, 4Z, 3XG
EOF

`awk` permite realizar sustituciones mediante el comando `gsub()` compuesto por una expresión regular que busca el patrón y el elemento que reemplazará o modificará el elemento encontrado. La `g`  indica que es global, en caso de que se requiera una sustitución local se debe utilizar la función `sub()`.



En el siguiente código se sustituye únicamente la primera ocurrencia en cada línea del archivo:

In [2]:
awk '{sub(/X/, "x"); print}' out.1

FieldA, fieldD, gieldE, FieldG
   2, x, 2X, 2XG
   2, Y, 2Y, 2YG 
   3, Y, 3Y, 3YG
   3, x, 3X, 3XG
   4, Z, 4Z, 3xG


Si se desean reemplazar todas las ocurrencias se usa `gsub`.

In [3]:
awk '{gsub(/X/, "x"); print}' out.1

FieldA, fieldD, gieldE, FieldG
   2, x, 2x, 2xG
   2, Y, 2Y, 2YG 
   3, Y, 3Y, 3YG
   3, x, 3x, 3xG
   4, Z, 4Z, 3xG


Se puede utiliza el comando `gsub` para varias sustituciones separandolo por **`;`** :

In [4]:
awk '{gsub(/X/, "x"); gsub(/Y/, "y"); print}' out.1

FieldA, fieldD, gieldE, FieldG
   2, x, 2x, 2xG
   2, y, 2y, 2yG 
   3, y, 3y, 3yG
   3, x, 3x, 3xG
   4, Z, 4Z, 3xG


---

La función `split` permite separar strings e incluir sus elementos en un array, el cual se recorre de acuerdo con la posición. Sigue la siguiente estructura:

      split($0,nombre_arreglo,"separador")


Sea el siguiente archivo:

In [43]:
cat > out.1 <<EOF
Maria-Patricia-Hernandez-Castro
Jose-Francisco-Montes-Lopez
EOF

El segundo nombre puede extraerse con:

In [44]:
awk '{split($0,nombre,"-")} {print nombre[2]}' out.1

Patricia
Francisco


---

A continuación se presenta un ejemplo más complejo de modificación de un patrón. Sea el siguiente archivo:

In [5]:
cat > out.1 <<EOF
Maria-1998:feb:2+M19
David-1972:nov:25+J45
Marco-2000:jun:4+V17
EOF

Se desea formatear la fecha de nacimiento completa de la persona y su edad, es decir, la primera linea:

```
Maria-1998:feb:2+M19
```

debe cambiarse por:

```
Maria 1998-02-02 M 19
```


La función `gensub` permite realizar busquedas a través de expresiones regulares y reemplazar dichos valores por elementos a elección de acuerdo con la frecuencia de coincidencia del patrón.


El primer paso consiste en reemplazar ` : `  por  `-`.

In [6]:
cat out.1

Maria-1998:feb:2+M19
David-1972:nov:25+J45
Marco-2000:jun:4+V17


In [21]:
### la función gensub solo funciona para la version gnu awk
### debe reemplazarse por la generica
awk '{print gensub(/\:([a-zA-Z]*)/,"-""\\1",1)}' out.1 > out.2
cat out.2

Maria-1998-feb:2+M19
David-1972-nov:25+J45
Marco-2000-jun:4+V17


La explicación del comando anterior es la siguiente:
 
 
Patrón entrada
  
 * El caracter `/`  indica el inicio y el fin de la expresión regular.
 * Los caracteres '(' y ')' contienen la expresión, caracter o dígito que se desea guardar.
 * [a-z] indica una cadena.
 * El caracter '*' indica que existen cadenas luego de la expresión regular
 
Patrón salida
 * "\\\1" primer elemento de la busqueda que fue guardado.
 
Frecuencia
 * "1" se debe reemplazar cuando encuentre el patrón por primera vez.

In [22]:
awk '{print gensub(/:([0-9])/, "-\\1", 1)}' out.2 > out.3
cat out.3

Maria-1998-feb-2+M19
David-1972-nov-25+J45
Marco-2000-jun-4+V17


In [23]:
awk '{print gensub(/-([0-9])*/, "-0\\1", 3)}' out.3 > out.4
cat out.4

Maria-1998-feb-02+M19
David-1972-nov-05+J45
Marco-2000-jun-04+V17


In [24]:
awk '{print gensub(/+([A-Z])([0-9][0-9])/, " \\1 \\2", 1)}' out.4 > out.5
cat out.5

Maria-1998-feb-02 M 19
David-1972-nov-05 J 45
Marco-2000-jun-04 V 17


In [25]:
awk '{sub(/-/, " "); print}' out.5 > out.6
cat out.6

Maria 1998-feb-02 M 19
David 1972-nov-05 J 45
Marco 2000-jun-04 V 17


In [26]:
awk '{gsub(/feb/, "02"); gsub(/nov/, "11");gsub(/jun/, "06");print}' out.6

Maria 1998-02-02 M 19
David 1972-11-05 J 45
Marco 2000-06-04 V 17


---

Sea el siguiente archivo:

In [ ]:
cat > out.1 <<EOF
Date, Price, Quantity, CustomerID
2013-01-12, 25, 7, 1
2014-05-12, 41, 5, 12
2013-02-25, 44, 3, 2
2013-04-04, 90, 1, 5
2013-06-21, 16, 2, 19
2014-05-12, 63, 2, 15
2014-05-12, 10, 4, 7
2013-02-28, 78, 8, 9
2013-08-02, 51, 1, 14
EOF

Se desea agregar un nuevo campo llamado `Quantity-CustomerID` que contenga la cantidad de producto y el cliente. El siguiente comando une las columnas de interés:

In [ ]:
awk '{print gensub(/, ([0-9][0-9]), ([0-9]), ([0-9])/, ", \\1, \\2-\\3", 1)}' out.1 > out.2
cat out.2

Se agrega el título `Quantity-CustomerID`:

In [ ]:
awk '{print gensub(/([a-zA-Z]*), ([a-zA-Z]*), ([a-zA-Z]*), ([a-zA-Z]*)/, "\\1, \\2, \\3-\\4", 1)}' out.2

A través del comando `BEGIN` y `END` se pueden agregar valores al principio y al final de las columnas o de todo el archivo de texto. 

El comando `NR>1` indica que se debe tener en cuenta solo filas a partir de la posicion 1 y `{print $0}` que se deben tener en cuenta todas las columnas.

In [ ]:
awk 'BEGIN{print "Date, Price, Quantity-CustomerID"}(NR>1){print $0}' out.2

In [ ]:
awk '(NR>1){print $3} END{print "Hola"}' out.2

* Se desea agregar un nuevo campo llamado `Quantity*Price` que contenga el total de la cuenta de cada compra. El comando {print `$1` `$2`} concatena e imprime columnas  iniciando el conteo en 1 de izquierda a derecha.

In [ ]:
awk -F"," '{print $1","$2","$3", "$4", "$2*$4}' out.1 > out.2
cat out.2

Ahora, se debe agregar el título a la nueva columna:

In [ ]:
## TODO: no funciona en BSD
awk '{print gsub(/, 0/, ", Total", 1)}' out.2 

Se puede utilizar el comando la opción `-F` y `OFS` para cambiar el separador del archivo de texto:

In [ ]:
awk -F"," 'BEGIN{OFS="|";}{print $1,$2,$3,$4}' out.1

---

**Ejercicio.--** Agregar una columna al archivo `USAA.txt` que contenga los años que hacen falta para que se acabe de pagar el crédito causado, a partir del año `2017`.

---

### Borrado de los archivos temporales creados

In [57]:
rm out.*

---

Edición de archivos con Awk
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia


---

Haga click [aquí](https://github.com/jdvelasq/bash-for-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/bash-for-analytics/tree/master/) para explorar el repositorio usando `nbviewer`. 